In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ppo import PPO
from config_utils import get_config
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
# import gym
# env = gym.make("CartPole-v1")
# env = gym.make("CarRacing-v0")
"""
observation = env.reset()
for _ in range(100):
  env.render()
  action = env.action_space.sample() # your agent here (this takes random actions)
  observation, reward, done, info = env.step(action)

  if done:
        break
    #observation = env.reset()
env.close()
"""

In [ ]:
from envs import env_carracing_simple
env = env_carracing_simple.CarRacingSimple()

In [ ]:
print(env.action_space)
print(env.action_space.shape)
print(env.observation_space)
print(env.observation_space.shape)
print(type(env.observation_space))

In [ ]:
from actor_critics import ac_utils
print(ac_utils.get_generic_space_size(env.action_space))

In [ ]:
config = get_config("configs/config-2020_06_02.json")
# config = get_config("configs/config-default_mujoco.json")

config.actor_critic["actor"]["learning_rate"] = 0.00005
config.actor_critic["critic"]["learning_rate"] = 0.00005
print(config)

# num_epochs = 1200
# num_epochs = 100
num_epochs = 2
minibatch_size = 1

if config.has_training:
    num_epochs = config.training.get("num_epochs", num_epochs)
    minibatch_size = config.training.get("minibatch_size", minibatch_size)

In [ ]:
from actor_critics import carracing_fc_ac
actor = carracing_fc_ac.Actor(env)
critic = carracing_fc_ac.Critic(env)

ppo = PPO(
    env,
    config=config,
    actor=actor,
    critic=critic,
    no_cuda=True
)

In [ ]:
loss_vals = []
pbar = tqdm(range(num_epochs))
for _ in pbar:
    #for _ in tqdm(range(minibatch_size), desc="minibatch"):
    step_loss_vals = ppo.step()
    loss_vals.append(sum(step_loss_vals) / len(step_loss_vals))
    tqdm.write(str(step_loss_vals))
    pbar.set_description("Loss = {:.6f}".format(loss_vals[-1]))

In [ ]:
plt.plot(loss_vals)
plt.xlabel("step")
plt.ylabel("loss")
plt.title("Training Loss Curve")
plt.show()

In [ ]:
import torch
from torch.autograd import Variable

In [ ]:
#env_new = wrappers.Monitor(env, './video/', force = True)
times = 0
env_new = env
observation = env_new.reset()
observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
for _ in range(100):
    env_new.render()
    action = torch.argmax(ppo.pi(observation.float())) 
    observation, reward, done, info = env_new.step(action.cpu().numpy())
    observation = Variable(torch.from_numpy(observation), requires_grad=False).to(ppo.torch_device)
    if not done:
        times += 1
        
print(times)
env_new.close()